# ESM

In [1]:
%cd ~/repo/protein-transfer

/home/t-fli/repo/protein-transfer


In [2]:
%load_ext blackcellmagic

In [3]:
from scr.preprocess.data_process import TaskProcess

In [4]:
TaskProcess().sum_file_df

,task,dataset,split,csv_path,fasta_path,pkl_path
0,annotation,scl,balanced,data/annotation/scl/balanced.csv,,
1,structure,secondary_structure,cb513,[data/structure/secondary_structure/secondary_...,,
2,proeng,gb1,two_vs_rest,data/proeng/gb1/two_vs_rest.csv,data/proeng/gb1/5LDE_1.fasta,data/proeng/gb1/two_vs_rest.pkl
3,proeng,gb1,low_vs_high,data/proeng/gb1/low_vs_high.csv,data/proeng/gb1/5LDE_1.fasta,data/proeng/gb1/low_vs_high.pkl
4,proeng,aav,one_vs_many,data/proeng/aav/one_vs_many.csv,data/proeng/aav/P03135.fasta,data/proeng/aav/one_vs_many.pkl
5,proeng,aav,two_vs_many,data/proeng/aav/two_vs_many.csv,data/proeng/aav/P03135.fasta,data/proeng/aav/two_vs_many.pkl
6,proeng,thermo,mixed,data/proeng/thermo/mixed.csv,,


In [2]:
import torch

In [3]:
model, alphabet = torch.hub.load("facebookresearch/esm:main", "esm1b_t33_650M_UR50S")

Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main


In [6]:
import lmdb

In [17]:
env = lmdb.open(str("data/structure/secondary_structure/secondary_structure_train.lmdb"), max_readers=1, readonly=True, lock=False, readahead=False, meminit=False)

In [19]:
import pickle as pkl

In [20]:
with env.begin(write=False) as txn:
    num_examples = pkl.loads(txn.get(b'num_examples'))

In [21]:
num_examples

8678

In [22]:
with env.begin(write=False) as txn:
    item = pkl.loads(txn.get(str(0).encode()))

In [24]:
item.keys()

dict_keys(['asa_max', 'disorder', 'evolutionary', 'interface', 'phi', 'primary', 'psi', 'rsa', 'ss3', 'ss8', 'valid_mask', 'id', 'protein_length'])

In [18]:
df_train = df.loc[(df["set"] == "train") & (df["validation"] != True)]
df_val = df.loc[(df["set"] == "train") & (df["validation"] == True)]
df_test = df.loc[(df["set"] == "test")]

len(df_train), len(df_val), len(df_test), len(df)

(4580, 509, 3644, 8733)

In [ ]:
tuple(zip(df_val.sequence, df_val.Code))

In [8]:
import numpy as np

In [6]:
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

# Prepare data (first 2 sequences from ESMStructuralSplitDataset superfamily / 4)
data = [
    ("2gi9", "MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE"),
]
batch_labels, batch_strs, batch_tokens = batch_converter(data)

# Extract per-residue representations (on CPU)
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[0], return_contacts=False)
token_representations = results["representations"][0]

# Generate per-sequence representations via averaging
# NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
sequence_representations = []
for i, (_, seq) in enumerate(data):
    print(token_representations[i, 1 : len(seq) + 1].shape)
    sequence_representations.append(token_representations[i, 1 : len(seq) + 1].mean(0))

print(token_representations, token_representations.shape, sequence_representations, len(sequence_representations[0]))

torch.Size([56, 1280])
tensor([[[-0.0663, -0.0206,  0.0212,  ...,  0.0336,  0.3534,  0.3684],
         [ 0.2793, -0.0092,  0.0531,  ...,  0.0053, -0.0854,  0.2145],
         [ 0.8682,  0.0139, -0.0363,  ..., -0.0775, -0.0688,  0.1010],
         ...,
         [-0.1451,  0.0310, -0.0578,  ..., -0.1140, -0.0157,  0.2026],
         [-0.0656, -0.0076, -0.0974,  ...,  0.0546,  0.0260,  0.2645],
         [-0.2940, -0.0431,  0.0488,  ...,  0.0070,  0.0056,  0.2847]]]) torch.Size([1, 58, 1280]) [tensor([-0.1665, -0.0073,  0.0005,  ...,  0.0174,  0.0038,  0.1164])] 1280


In [8]:

# Extract per-residue representations (on CPU)
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[0], return_contacts=False)
token_representations = results["representations"][0]

# Generate per-sequence representations via averaging
# NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
sequence_representations = []
for i, (_, seq) in enumerate(data):
    print(token_representations[i, 1 : len(seq) + 1].shape)
    sequence_representations.append(token_representations[i, 1 : len(seq) + 1].mean(0))

print(token_representations, token_representations.shape, sequence_representations, len(sequence_representations[0]))

torch.Size([56, 1280])
tensor([[[-0.0663, -0.0206,  0.0212,  ...,  0.0336,  0.3534,  0.3684],
         [ 0.2793, -0.0092,  0.0531,  ...,  0.0053, -0.0854,  0.2145],
         [ 0.8682,  0.0139, -0.0363,  ..., -0.0775, -0.0688,  0.1010],
         ...,
         [-0.1451,  0.0310, -0.0578,  ..., -0.1140, -0.0157,  0.2026],
         [-0.0656, -0.0076, -0.0974,  ...,  0.0546,  0.0260,  0.2645],
         [-0.2940, -0.0431,  0.0488,  ...,  0.0070,  0.0056,  0.2847]]]) torch.Size([1, 58, 1280]) [tensor([-0.1665, -0.0073,  0.0005,  ...,  0.0174,  0.0038,  0.1164])] 1280


In [3]:
from scr.encoding.encoding_classes import ESMEncoder

In [5]:
next(ESMEncoder(
    esm_model_name="esm1b_t33_650M_UR50S",
    esm_emb_layer=0,
    fasta_loc="",
    fasta_seq="MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE",
).encode(mut_seqs="MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE"))

Loading ESM model: esm1b_t33_650M_UR50S


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main


No embedding flattening


array([[[ 0.27928993, -0.00919111,  0.05307703, ...,  0.0052783 ,
         -0.08536895,  0.2144556 ],
        [ 0.86820096,  0.01390768, -0.03625386, ..., -0.07747608,
         -0.06879894,  0.10098855],
        [ 0.9904541 ,  0.02066673,  0.04346693, ..., -0.04951385,
          0.04328922, -0.09477672],
        ...,
        [ 0.07968199, -0.01003133, -0.02964539, ...,  0.00346076,
         -0.45212165,  0.23354846],
        [-0.14514625,  0.03099104, -0.05777264, ..., -0.11398804,
         -0.01572975,  0.20261148],
        [-0.06561151, -0.00760958, -0.09738796, ...,  0.05461294,
          0.02598158,  0.26446784]]], dtype=float32)

In [6]:
next(ESMEncoder(
    esm_model_name="esm1b_t33_650M_UR50S",
    esm_emb_layer=0,
    fasta_loc="",
    fasta_seq="MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE",
).encode(mut_seqs="MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE",flatten_emb="mean"))

Loading ESM model: esm1b_t33_650M_UR50S


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main


array([[-0.16647258, -0.00734616,  0.00054652, ...,  0.01738006,
         0.00381861,  0.11643288]], dtype=float32)

In [9]:

# Extract per-residue representations (on CPU)
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[1], return_contacts=False)
token_representations = results["representations"][1]

# Generate per-sequence representations via averaging
# NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
sequence_representations = []
for i, (_, seq) in enumerate(data):
    print(token_representations[i, 1 : len(seq) + 1].shape)
    sequence_representations.append(token_representations[i, 1 : len(seq) + 1].mean(0))

print(token_representations, token_representations.shape, sequence_representations, len(sequence_representations[0]))

torch.Size([56, 1280])
tensor([[[-0.3899,  0.1210, -0.7449,  ..., -0.9544,  0.4704,  0.6206],
         [ 0.4447,  0.8576,  0.1733,  ..., -0.8428,  0.4788,  0.9012],
         [ 1.3972,  1.1721, -0.5245,  ..., -1.4560,  0.0084,  0.3406],
         ...,
         [ 0.4156,  0.8773,  0.4632,  ..., -0.9489, -0.3087,  1.6864],
         [ 0.7929,  0.9090, -0.9665,  ..., -0.6917,  0.5742,  0.9072],
         [-0.0441,  0.6803, -0.3759,  ..., -0.3495,  0.1413,  0.8631]]]) torch.Size([1, 58, 1280]) [tensor([ 0.1365,  0.7353, -0.1419,  ..., -0.5605,  0.1081,  0.6370])] 1280


In [7]:
next(ESMEncoder(
    esm_model_name="esm1b_t33_650M_UR50S",
    esm_emb_layer=1,
    fasta_loc="",
    fasta_seq="MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE",
).encode(mut_seqs="MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE",flatten_emb="mean"))

Loading ESM model: esm1b_t33_650M_UR50S


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main


array([[ 0.13647312,  0.7353482 , -0.14194234, ..., -0.5604886 ,
         0.10806996,  0.6370432 ]], dtype=float32)

In [19]:
# Extract per-residue representations (on CPU)
with torch.no_grad():
    results = model(batch_tokens, repr_layers=list(range(0, 33+1)), return_contacts=False)
print(results["representations"].keys())
token_representations = results["representations"][1]
print(token_representations)


dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33])
tensor([[[-0.3899,  0.1210, -0.7449,  ..., -0.9544,  0.4704,  0.6206],
         [ 0.4447,  0.8576,  0.1733,  ..., -0.8428,  0.4788,  0.9012],
         [ 1.3972,  1.1721, -0.5245,  ..., -1.4560,  0.0084,  0.3406],
         ...,
         [ 0.4156,  0.8773,  0.4632,  ..., -0.9489, -0.3087,  1.6864],
         [ 0.7929,  0.9090, -0.9665,  ..., -0.6917,  0.5742,  0.9072],
         [-0.0441,  0.6803, -0.3759,  ..., -0.3495,  0.1413,  0.8631]]])


In [6]:
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

# Prepare data (first 2 sequences from ESMStructuralSplitDataset superfamily / 4)
data = [
    ("2gi9", "MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE"),
]
batch_labels, batch_strs, batch_tokens = batch_converter(data)

# Extract per-residue representations (on CPU)
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[33], return_contacts=False)
token_representations = results["representations"][33]

# Generate per-sequence representations via averaging
# NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
sequence_representations = []
for i, (_, seq) in enumerate(data):
    sequence_representations.append(token_representations[i, 1 : len(seq) + 1].mean(0))

print(token_representations, token_representations.shape, sequence_representations, len(sequence_representations[0]))

tensor([[[-0.0454,  0.2249,  0.1654,  ...,  0.0380,  0.0451, -0.0621],
         [-0.0642, -0.2746,  0.4808,  ..., -0.0545, -0.1764,  0.0235],
         [-0.1702,  0.0312, -0.2013,  ..., -0.0612, -0.2743,  0.3677],
         ...,
         [-0.0621,  0.2908,  0.6940,  ..., -0.2058, -0.0074,  0.4298],
         [-0.1076,  0.1801,  0.1223,  ..., -0.1654,  0.0073,  0.0412],
         [-0.0576, -0.0605,  0.0807,  ..., -0.0661,  0.1188,  0.1108]]]) torch.Size([1, 58, 1280]) [tensor([-0.0211,  0.1367, -0.0039,  ..., -0.0647, -0.0582,  0.1644])] 1280


In [9]:
next(ESMEncoder(
    esm_model_name="esm1b_t33_650M_UR50S",
    esm_emb_layer=33,
    fasta_loc="",
    fasta_seq="MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE",
).encode(mut_seqs="MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE",flatten_emb="mean"))

Loading ESM model: esm1b_t33_650M_UR50S


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main


array([[-0.02112344,  0.1367362 , -0.00392063, ..., -0.06472861,
        -0.0582489 ,  0.16439287]], dtype=float32)